<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/1_transformer/gp-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer 실습
참고 : [Attention is All You Need Tutorial(ndb796)](https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb)

자연어처리 끝판왕. 다른 분야에도 고르게 사용됨.

## 자연어를 어떻게 Input으로 줄 수 있을까?
기본적으로 모델의 INPUT은 숫자 형태로 들어가기 때문에 처리하고자 하는 문장을 분해 및 숫자로 변환하는 전처리 작업이 필요하다. 이 과정을 문장의 토큰화(tokenization)이라고 한다.

**torchtext**는 자연어처리에 유용한 라이브러리로, 데이터셋과 전처리 클래스, 스코어링 함수 등이 존재한다. 설치해보자!

In [4]:
%%capture
!pip install torchtext==0.6.0

NotImplementedError: ignored

이번에는 **spaCy** 라이브러리를 이용해보자. **토큰화(tokenization), 태깅(tagging) 등의 전처리**를 위한 라이브러리다. 영어와 한국어 전처리 모듈을 설치하고 확인해보자

In [5]:
%%capture
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm

NotImplementedError: ignored

In [6]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_ko = spacy.load('ko_core_news_sm') # 한국어 토큰화(tokenization)

In [7]:
# 간단히 토큰화(tokenization) 기능 써보기
_tmp_en_tokens = spacy_en.tokenizer("I graduated last week.")
_tmp_ko_tokens = spacy_ko.tokenizer("저는 저번 주에 졸업했습니다.")

# 영어 문장 토큰화
for i, token in enumerate(_tmp_en_tokens):
    print(f"인덱스 {i}: {token.text}")

print("--------------")

# 한국어 문장 토큰화
for i, token in enumerate(_tmp_ko_tokens):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: graduated
인덱스 2: last
인덱스 3: week
인덱스 4: .
--------------
인덱스 0: 저는
인덱스 1: 저번
인덱스 2: 주에
인덱스 3: 졸업했습니다
인덱스 4: .


토큰화 과정을 함수화 해보자

In [8]:
tokenizer_en = lambda text: [token.text for token in spacy_en.tokenizer(text)]
tokenizer_ko = lambda text: [token.text for token in spacy_ko.tokenizer(text)]

In [9]:
tokenizer_en('I graduated last week.')

['I', 'graduated', 'last', 'week', '.']

필드(field)를 이용해 데이터셋에 대한 구체적인 전처리 과정을 명시할 수 있다. 이 작업을 통해 추후 코드에서 전처리를 조금 더 단순하게 다룰 수 있다.

In [90]:
from torchtext.data import Field

# 목표 : ko -> en 로 번역
SRC = Field(tokenize=tokenizer_ko, init_token="", eos_token="", lower=True, batch_first=True)
DST = Field(tokenize=tokenizer_en, init_token="", eos_token="", lower=True, batch_first=True)

**Parameter**
- tokenize : 어떤 토큰화 함수를 적용할 것인지 지정(default : split)
- init_token : ?
- eos_token : ?
- lower : 대문자가 있다면 소문자로 변경할 것인지
- batch_first : ?(미니배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지)

이제 데이터셋을 불러오자. **torchtext의 datasets**에는 대표적인 번역 데이터셋인 **Mullti30k**이 있다. 였는데,,, 아쉽게도 한국어는 없어서 다른 라이브러리를 사용함.

여러 데이터셋을 모아서 라이브러리화 하는 **Korpora**라는 라이브러리가 있었음

In [11]:
# 아,,, colab에서 UTF-8 오류 뜨는데 아래 코드로 해결함,,, 찝찝,,,,하네
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [13]:
%%capture
!pip install Korpora

In [44]:
from Korpora import Korpora
Korpora.fetch("korean_parallel_koen_news")
datasets = Korpora.load("korean_parallel_koen_news")

[Korpora] Corpus `korean_parallel` is already installed at /root/Korpora/korean_parallel/korean-english-park.train.tar.gz
decompress /root/Korpora/korean_parallel/korean-english-park.train.tar.gz
[Korpora] Corpus `korean_parallel` is already installed at /root/Korpora/korean_parallel/korean-english-park.dev.tar.gz
decompress /root/Korpora/korean_parallel/korean-english-park.dev.tar.gz
[Korpora] Corpus `korean_parallel` is already installed at /root/Korpora/korean_parallel/korean-english-park.test.tar.gz
decompress /root/Korpora/korean_parallel/korean-english-park.test.tar.gz

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : KakaoBrain
    Repository : https://github.com/jungyeul/korean-parallel-corpora
    References :
        - Jungyeul Park, Jeen-Pyo Hong and Jeon

In [67]:
datasets.train

koennews.train: size=94123
  - koennews.train.texts : list[str]
  - koennews.train.pairs : list[str]

In [68]:
datasets.test

koennews.test: size=2000
  - koennews.test.texts : list[str]
  - koennews.test.pairs : list[str]

In [70]:
import torchtext
torchtext.datasets

<module 'torchtext.datasets' from '/usr/local/lib/python3.8/dist-packages/torchtext/datasets/__init__.py'>

In [73]:
train_data, test_data = TabularDataset.splits(
        path='.', train=datasets.train, test=datasets.test, format=,
        fields=[('text', SRC), ('label', DST)], skip_header=True)

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__path__',
 '__file__',
 '__cached__',
 '__builtins__',
 'utils',
 'vocab',
 'data',
 'datasets',
 'experimental',
 '__version__',
 '__all__']

In [102]:
train_dataset = torchtext.data.Dataset(
                    examples=[datasets.train.texts, datasets.train.pairs],
                    fields=[('src', SRC), ('trg', DST)])
test_dataset  = torchtext.data.Dataset(
                    examples=[datasets.test.texts, datasets.test.pairs],
                    fields=[('src', SRC), ('trg', DST)])

In [110]:
test = torchtext.data.Dataset(
                    examples=datasets.train.texts,
                    fields=[('src', SRC)])

In [111]:
test.examples[20]

'북한의 핵무기 계획을 포기하도록 하려는 압력이 거세지고 있는 가운데, 일본과 북한의 외교관들이 외교 관계를 정상화하려는 회담을 재개했다.'

In [ ]:
torchtext.data.TabularDataset()